<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_tower_ta_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
df0 = pd.read_csv(fn, parse_dates=["measured_at"]).dropna(how="all", axis=1)
df0

In [ ]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11 & net_type == 'LTE'").copy()
df1

In [ ]:
# 日時変換
df1["measured_at"] = df1["measured_at"].dt.tz_convert("Asia/Tokyo").dt.tz_localize(None)

In [ ]:
# 1行ずらす
df1["ta_before"] = df1["ta"].shift(1)
df1["dt_before"] = df1["measured_at"].shift(1)

In [ ]:
# 時間差分の秒数
df1["time"] = (df1["measured_at"] - df1["dt_before"]).dt.total_seconds()

In [ ]:
# セル変異を抽出
df2 = df1.loc[df1["cell_id"] != df1["cell_id"].shift(1), ["cell_id", "time", "ta", "ta_before"]]

In [ ]:
df2.shape

In [ ]:
# 欠損除去
df3 = df2.dropna(how="any")

In [ ]:
df3.shape

In [ ]:
df3

In [ ]:
# 同じ
(df3["ta"] == df3["ta_before"]).sum()

In [ ]:
# 違う
(df3["ta"] != df3["ta_before"]).sum()

In [ ]:
1# TAが違うものを抽出
df4 = df3[df3["ta"] != df3["ta_before"]]

In [ ]:
# グラフ表示
df4["time"].round(1).value_counts().sort_index().plot.bar(figsize=(20,5))